In [4]:
import analyzer
import time
import pandas as pd
%run analyzer.py mnist_nets/mnist_relu_3_10.txt mnist_images/img0.txt 0.0005

Academic license - for non-commercial use only
verified
analysis time:  0.45993852615356445  seconds


In [263]:
def paper_analyze(netname, epsilon, specname, iobj= None):
    verified = False
    #Import and parse the net
    with open(netname, 'r') as netfile:
        netstring = netfile.read()
    with open(specname, 'r') as specfile:
        specstring = specfile.read()
    nn = parse_net(netstring)
    x0_low, x0_high = parse_spec(specstring)
    x = x0_low[1:]
    xl, xu = get_perturbed_image(x0_low,0)
    label, _ = analyze(nn,xl, xu,0)
    xl, xu = get_perturbed_image(x0_low,epsilon)
    assert(xl.shape == xu.shape)
    
    # Initialization. 
    v_hat_1 = nn.weights[0].T
    gamma_1 = nn.biases[0].T
    wnorm = np.linalg.norm(nn.weights[0].T, axis=0, ord=1)
    l_i = np.matmul(np.transpose(x), nn.weights[0].T) + nn.biases[0] - epsilon*wnorm
    u_i = np.matmul(np.transpose(x), nn.weights[0].T) + nn.biases[0] + epsilon*wnorm
    
    vs = [None]
    gammas = [gamma_1]
    ls = [l_i]
    
    # Iteration through all layer. 
    for i in range(2, nn.numlayer): 
        print("Layer -- %d" % i)
        # Check valid bounds. 
        assert (l_i <= u_i).all()
        num_neurons = nn.biases[i-1].shape[0]
        # Form index subset of lower and bounds
        # I_im = both lower and upper bound are negativ.
        # I_ip = both lower and upper bound are positiv. 
        # I_i  = lower bound negativ, upper bound positiv.
        I_im = np.intersect1d(np.where(l_i < 0)[0], np.where(u_i < 0)[0])
        I_ip = np.intersect1d(np.where(l_i >= 0)[0], np.where(u_i >= 0)[0])
        I_i  = np.intersect1d(np.where(l_i < 0)[0], np.where(u_i > 0)[0])
        assert I_im.shape[0] + I_ip.shape[0] + I_i.shape[0] == num_neurons
        # Build diagonal bounds matrix. 
        # D_jj = 0 if j in I_im. 
        # D_jj = 1 if j in I_ip. 
        # D_jj = u_j/(u_j - l_j) if j in I_i.
        D_i = np.zeros((num_neurons,num_neurons))
        D_i[I_ip,I_ip] = 1
        D_i[I_i,I_i] = u_i[I_i]/(u_i[I_i] - l_i[I_i])
        # Initialize new terms.
        v_iIi = np.matmul(D_i[I_i], nn.weights[i].T)
        gamma_i = nn.biases[i].T
        # Propagate existing terms. 
        for j in range(2, i): 
            vs[j-1] = np.matmul(np.matmul(vs[j-1], D_i), nn.weights[i].T)
        for j in range(1, i): 
            gammas[j-1] = np.matmul(gammas[j-1],np.matmul(D_i, nn.weights[i].T))
        v_hat_1 = np.matmul(v_hat_1, np.matmul(D_i, nn.weights[i].T))
        # Update internal storage. 
        vs.append(v_iIi)
        gammas.append(gamma_i)
        ls.append(l_i)        
        # Compute bounds.
        psi_i = np.matmul(x.T, v_hat_1) + sum(gammas)
        vhatnorm = np.linalg.norm(v_hat_1, axis=0, ord=1)
        l_i = psi_i - epsilon*vhatnorm
        for j in range(2,i+1): 
            for it in I_i.tolist(): 
                l_i += ls[j-1][it]*np.maximum(vs[j-1][0,it],0)
        u_i = psi_i + epsilon*vhatnorm
        for j in range(2,i+1): 
            for it in I_i.tolist(): 
                u_i -= ls[j-1][it]*np.maximum(vs[j-1][0,it],0)
        print(l_i, u_i)
    return flag
    

In [264]:
netname = 'mnist_nets/mnist_relu_6_50.txt'
epsilon = 0.01
specname = 'mnist_images/img1.txt'

In [265]:
%%time
label = paper_analyze(netname, epsilon, specname)

Layer -- 2
[ 0.66296313  0.38913563 -0.3218351  -0.76884942 -1.18706421 -0.49741311
 -1.20088873 -1.21607726  1.2984405  -2.64780469 -2.27276845  0.52549433
 -0.51101516  0.260106    1.06149689 -0.04746931 -2.0880292   0.33782916
 -1.68352757 -0.22303646 -0.90345151 -1.3282468   0.64325599 -0.51584231
  0.45563959  0.41923451 -1.70856223 -0.24263303 -2.24619574  1.02085667
  0.09093268 -0.6798925   0.13746135 -1.40323514  0.50509378 -1.27519787
  1.09443297 -2.54913317  0.53274823  0.25060667 -0.6290629  -0.07649312
  0.61085044  0.3300804   0.17188211 -1.16317163 -2.54895849  1.65023952
 -0.59936431 -0.04824073] [ 1.12884449  1.16810607  0.28309485 -0.17242852 -0.57750647  0.00497402
 -0.48251956 -0.36595092  2.01782852 -1.90866261 -1.54752874  1.20438815
 -0.04692598  1.17977138  1.859357    0.97298068 -1.26191093  0.84908984
 -1.0440384   0.62640434 -0.33521956 -0.77465494  1.77345918  0.11227872
  1.30661242  1.14340295 -0.81579296  0.47110673 -1.57400828  1.94559014
  0.64234067 -

IndexError: index 10 is out of bounds for axis 1 with size 10